In [9]:
# Imports

import pandas as pd
import os
from datetime import datetime

In [10]:
# Dossiers

RAW_DIR = "../data/raw"
PROCESSED_DIR = "../data/processed/"


In [11]:
# Dictionnaire des fichiers

files = {
    "student_info": "studentInfo.csv",
    "courses": "courses.csv",
    "registrations": "studentRegistration.csv",
    "vle_info": "vle.csv",
    "assessments": "assessments.csv",
    "student_vle": "studentVle.csv",
    "student_assessment": "studentAssessment.csv"
}
for name, fname in files.items():
    path = os.path.join(RAW_DIR, fname)
    print(f"{name}: {path} -> exists? {os.path.exists(path)}")

student_info: ../data/raw\studentInfo.csv -> exists? True
courses: ../data/raw\courses.csv -> exists? True
registrations: ../data/raw\studentRegistration.csv -> exists? True
vle_info: ../data/raw\vle.csv -> exists? True
assessments: ../data/raw\assessments.csv -> exists? True
student_vle: ../data/raw\studentVle.csv -> exists? True
student_assessment: ../data/raw\studentAssessment.csv -> exists? True


In [12]:
data = {}
for name, fname in files.items():
    path = os.path.join(RAW_DIR, fname)
    if os.path.exists(path):
        data[name] = pd.read_csv(path)
        print(f"{name} chargé avec {len(data[name])} lignes")
    else:
        print(f"ERREUR: {name} ({fname}) non trouvé à {RAW_DIR}")

# Accès facile aux DataFrames
student_info = data["student_info"]
courses = data["courses"]
registrations = data["registrations"]
vle = data["vle_info"]
assessments = data["assessments"]
student_vle = data["student_vle"]
student_assessment = data["student_assessment"]

student_info chargé avec 32593 lignes
courses chargé avec 22 lignes
registrations chargé avec 32593 lignes
vle_info chargé avec 6364 lignes
assessments chargé avec 206 lignes
student_vle chargé avec 10655280 lignes
student_assessment chargé avec 173912 lignes


In [14]:
def safe_str(value):
    if pd.isna(value):
        return None
    s = str(value).strip()
    return s if s else None

def safe_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

def safe_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def safe_date(value):
    if pd.isna(value) or value is None:
        return None
    try:
        d = pd.to_datetime(value, errors="coerce")
        if pd.isna(d):
            return None
        return d.strftime("%Y-%m-%d")
    except:
        return None

def days_since_1970_to_date(n):
    try:
        d = pd.Timestamp("1970-01-01") + pd.Timedelta(days=int(n))
        return d.strftime("%Y-%m-%d")
    except:
        return None


# =========================================
# NORMALISATION DES TABLES
# =========================================

def normalize_student_info(df):
    df = df.copy()
    df["student_id"] = df["id_student"].apply(safe_int)
    df["code_module"] = df["code_module"].apply(safe_str)
    df["code_presentation"] = df["code_presentation"].apply(safe_str)
    df["gender"] = df["gender"].apply(safe_str)
    df["region"] = df["region"].apply(safe_str)
    df["highest_education"] = df["highest_education"].apply(safe_str)
    df["imd_band"] = df["imd_band"].apply(safe_str)
    df["age_band"] = df["age_band"].apply(safe_str)
    df["num_of_prev_attempts"] = df["num_of_prev_attempts"].apply(safe_int)
    df["studied_credits"] = df["studied_credits"].apply(safe_int)
    df["disability"] = df["disability"].apply(safe_str)
    df["final_result"] = df["final_result"].apply(safe_str)

    return df[[
        "student_id","code_module","code_presentation","gender","region",
        "highest_education","imd_band","age_band","num_of_prev_attempts",
        "studied_credits","disability","final_result"
    ]]


def normalize_courses(df):
    df = df.copy()
    df["code_module"] = df["code_module"].apply(safe_str)
    df["code_presentation"] = df["code_presentation"].apply(safe_str)
    df["module_presentation_length"] = df["module_presentation_length"].apply(safe_int)

    df = df[df["code_module"].notna() & df["code_presentation"].notna()]
    return df[["code_module","code_presentation","module_presentation_length"]]


def normalize_registrations(df):
    df = df.copy()
    df["student_id"] = df["id_student"].apply(safe_int)
    df["code_module"] = df["code_module"].apply(safe_str)
    df["code_presentation"] = df["code_presentation"].apply(safe_str)
    df["date_registration"] = df["date_registration"].apply(safe_date)
    df["date_unregistration"] = df["date_unregistration"].apply(safe_date)

    df = df[df["student_id"].notna()]
    return df[["student_id","code_module","code_presentation","date_registration","date_unregistration"]]


def normalize_vle_info(df):
    df = df.copy()
    df["id_site"] = df["id_site"].apply(safe_int)
    df["code_module"] = df["code_module"].apply(safe_str)
    df["code_presentation"] = df["code_presentation"].apply(safe_str)
    df["activity_type"] = df["activity_type"].apply(safe_str)
    df["week_from"] = df["week_from"].apply(safe_int)
    df["week_to"] = df["week_to"].apply(safe_int)

    return df[["id_site","code_module","code_presentation","activity_type","week_from","week_to"]]


def normalize_assessments(df):
    df = df.copy()
    df["code_module"] = df["code_module"].apply(safe_str)
    df["code_presentation"] = df["code_presentation"].apply(safe_str)
    df["id_assessment"] = df["id_assessment"].apply(safe_int)
    df["assessment_type"] = df["assessment_type"].apply(safe_str)
    df["assessment_date"] = df["date"].apply(safe_date)
    df["weight"] = df["weight"].apply(safe_float)

    return df[["code_module","code_presentation","id_assessment","assessment_type","assessment_date","weight"]]


def normalize_student_vle(df):
    df = df.copy()
    date_col = "date" if "date" in df.columns else "activity_date"
    df["id_student"] = df["id_student"].apply(safe_int)
    df["code_module"] = df["code_module"].apply(safe_str)
    df["code_presentation"] = df["code_presentation"].apply(safe_str)
    df["id_site"] = df["id_site"].apply(safe_int)
    df["activity_date"] = df[date_col].apply(safe_date)
    df["sum_click"] = df["sum_click"].apply(safe_int)
    df["activity_type"] = "VLE_CLICK"

    df = df.dropna()
    df = df.drop_duplicates(subset=["id_student","code_module","code_presentation","id_site","activity_date"])
    return df[["id_student","code_module","code_presentation","id_site","activity_date","sum_click","activity_type"]]


def normalize_student_assessment(df):
    df = df.copy()
    df["student_id"] = df["id_student"].apply(safe_int)
    df["activity_date"] = df["date_submitted"].apply(days_since_1970_to_date)
    df["activity_type"] = "ASSESSMENT"
    df["score"] = df["score"].apply(safe_float)
    df["assessment_type"] = df.get("assessment_type","exam")
    return df[["student_id","activity_date","activity_type","score","assessment_type"]]

In [15]:
# Normalisation de toutes les tables
students_normalized = normalize_student_info(student_info)
courses_normalized = normalize_courses(courses)
registrations_normalized = normalize_registrations(registrations)
vle_normalized = normalize_vle_info(vle)
assessments_normalized = normalize_assessments(assessments)
student_vle_limited = student_vle.head(40000)
student_vle_normalized = normalize_student_vle(student_vle_limited)
student_assessment_normalized = normalize_student_assessment(student_assessment)

# Affichage des 10 premières lignes
print("=== Students ===")
print(students_normalized.head(10))

print("\n=== Courses ===")
print(courses_normalized.head(10))

print("\n=== Registrations ===")
print(registrations_normalized.head(10))

print("\n=== VLE Info ===")
print(vle_normalized.head(10))

print("\n=== Assessments ===")
print(assessments_normalized.head(10))

print("\n=== Student VLE ===")
print(student_vle_normalized.head(10))

print("\n=== Student Assessment ===")
print(student_assessment_normalized.head(10))

=== Students ===
   student_id code_module code_presentation gender                region  \
0       11391         AAA             2013J      M   East Anglian Region   
1       28400         AAA             2013J      F              Scotland   
2       30268         AAA             2013J      F  North Western Region   
3       31604         AAA             2013J      F     South East Region   
4       32885         AAA             2013J      F  West Midlands Region   
5       38053         AAA             2013J      M                 Wales   
6       45462         AAA             2013J      M              Scotland   
7       45642         AAA             2013J      F  North Western Region   
8       52130         AAA             2013J      F   East Anglian Region   
9       53025         AAA             2013J      M          North Region   

             highest_education imd_band age_band  num_of_prev_attempts  \
0             HE Qualification  90-100%     55<=                     0  

In [16]:
# Sauvegarder chaque table normalisée
students_normalized.to_csv(os.path.join(PROCESSED_DIR, "student_info_normalized.csv"), index=False)
courses_normalized.to_csv(os.path.join(PROCESSED_DIR, "courses_normalized.csv"), index=False)
registrations_normalized.to_csv(os.path.join(PROCESSED_DIR, "registrations_normalized.csv"), index=False)
vle_normalized.to_csv(os.path.join(PROCESSED_DIR, "vle_info_normalized.csv"), index=False)
assessments_normalized.to_csv(os.path.join(PROCESSED_DIR, "assessments_normalized.csv"), index=False)
student_vle_normalized.to_csv(os.path.join(PROCESSED_DIR, "student_vle_normalized.csv"), index=False)
student_assessment_normalized.to_csv(os.path.join(PROCESSED_DIR, "student_assessment_normalized.csv"), index=False)

print("✅ Toutes les tables normalisées ont été sauvegardées dans", PROCESSED_DIR)


✅ Toutes les tables normalisées ont été sauvegardées dans ../data/processed/
